In [1]:
# Put your KFP cluster endpoint URL here if working from GCP notebooks (or local notebooks). ('https://xxxxx.notebooks.googleusercontent.com/')
kfp_endpoint='https://kubeflow-api.k8s.tidu.giize.com/pipeline'
# CA_BUNDLE='/etc/ssl/certs/ca-certificates.crt'

In [2]:
from typing import NamedTuple

import kfp
from kfp.components import InputPath, InputTextFile, OutputPath, OutputTextFile
from kfp.components import func_to_container_op

In [3]:
client = kfp.Client(host=kfp_endpoint)

In [30]:
client.list_pipelines()

{'next_page_token': None,
 'pipelines': [{'created_at': datetime.datetime(2022, 2, 15, 2, 52, 20, tzinfo=tzlocal()),
                'default_version': {'code_source_url': None,
                                    'created_at': datetime.datetime(2022, 2, 15, 2, 52, 20, tzinfo=tzlocal()),
                                    'description': None,
                                    'id': '9526b615-51ae-44dc-a143-df3c01a0c40c',
                                    'name': '[Demo] XGBoost - Iterative model '
                                            'training',
                                    'package_url': None,
                                    'parameters': None,
                                    'resource_references': [{'key': {'id': '9526b615-51ae-44dc-a143-df3c01a0c40c',
                                                                     'type': 'PIPELINE'},
                                                             'name': None,
                                           

In [4]:
client.list_runs()

{'next_page_token': None,
 'runs': [{'created_at': datetime.datetime(2022, 2, 15, 2, 54, 22, tzinfo=tzlocal()),
           'description': None,
           'error': None,
           'finished_at': datetime.datetime(2022, 2, 15, 3, 6, 29, tzinfo=tzlocal()),
           'id': '21778d59-6fca-4af3-a3d3-4b6ed094c0f3',
           'metrics': None,
           'name': 'Run of [Demo] XGBoost - Iterative model training (2a905)',
           'pipeline_spec': {'parameters': None,
                             'pipeline_id': None,
                             'pipeline_manifest': None,
                             'pipeline_name': None,
                             'runtime_config': None,
                             'workflow_manifest': '{"kind":"Workflow","apiVersion":"argoproj.io/v1alpha1","metadata":{"generateName":"train-until-good-pipeline-","creationTimestamp":null,"labels":{"pipelines.kubeflow.org/kfp_sdk_version":"1.5.1"},"annotations":{"pipelines.kubeflow.org/kfp_sdk_version":"1.5.1","pipeline

In [8]:
client.delete_experiment('348e6f67-401a-458e-bedf-aab3855fe269')

{}

In [46]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2022, 2, 15, 2, 54, 10, tzinfo=tzlocal()),
                  'description': None,
                  'id': '0cc9e029-e0db-499a-86fb-dcad27e48e03',
                  'name': 'Default',
                  'resource_references': [{'key': {'id': 'tidu-nguyen-2000',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}

In [47]:
@func_to_container_op
def print_small_text(text: str):
    '''Print small text'''
    print(text)

def constant_to_consumer_pipeline():
    '''Pipeline that passes small constant string to to consumer'''
    consume_task = print_small_text('Hello world') # Passing constant as argument to consumer

client.create_run_from_pipeline_func(constant_to_consumer_pipeline, arguments={})

RunPipelineResult(run_id=cb887cd9-3b35-4f99-8132-be3af908b21a)

In [54]:
@func_to_container_op
def markdown_vis(mlpipeline_ui_metadata_path: kfp.components.OutputPath()):
  import json
    
  metadata = {
    'outputs' : [
    # Markdown that is hardcoded inline
    {
      'storage': 'inline',
      'source': '# Inline Markdown\n[A link](https://www.kubeflow.org/)',
      'type': 'markdown',
    }]
  }

  with open(mlpipeline_ui_metadata_path, 'w') as metadata_file:
    json.dump(metadata, metadata_file)

In [66]:
def roc_vis(roc_csv_file_path: str, mlpipeline_ui_metadata_path: kfp.components.OutputPath()):
  import json
  import pandas as pd

  df_roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'thresholds': thresholds})
  roc_file = os.path.join(roc_csv_file_path, 'roc.csv')
  with file_io.FileIO(roc_file, 'w') as f:
    df_roc.to_csv(f, columns=['fpr', 'tpr', 'thresholds'], header=False, index=False)

  metadata = {
    'outputs': [{
      'type': 'roc',
      'format': 'csv',
      'schema': [
        {'name': 'fpr', 'type': 'NUMBER'},
        {'name': 'tpr', 'type': 'NUMBER'},
        {'name': 'thresholds', 'type': 'NUMBER'},
      ],
      'source': roc_file
    }]
  }

  with open(mlpipeline_ui_metadata_path, 'w') as metadata_file:
    json.dump(metadata, metadata_file)

In [67]:
def test_visualization():
  markdown_vis()
  func_to_container_op(
    roc_vis,
    packages_to_install=['pandas'],
    base_image='python:3.9'
    )('/tmp')
client.create_run_from_pipeline_func(test_visualization, arguments={})

RunPipelineResult(run_id=ab97d6e0-65fa-4c89-891e-ba650c3ff927)